In [ ]:
!pip install langchain sentence-transformers chromadb llama-cpp-python langchain_community pypdf

In [2]:
from google.colab import userdata
import os
from getpass import getpass


HF_API_KEY = userdata.get("HF_API_KEY")
os.environ["HF_API_KEY"] = "HF_API_KEY"

In [3]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain
import pathlib
import textwrap
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [4]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

In [5]:
loader = DirectoryLoader(
    '/content/drive/My Drive/medical_data',
    glob="**/*.txt",
    loader_cls=TextLoader
)
docs = loader.load()

In [ ]:
loader = PyPDFDirectoryLoader('/content/drive/My Drive/medical_data')
docs = loader.load()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

In [ ]:
# Create Chroma vector store
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

# Test query
query = "who is at risk of heart disease"
search = vectorstore.similarity_search(query, k=3)
print(search[0].page_content)

In [ ]:
retriever = vectorstore.as_retriever(
    search_kwargs={'k': 5}
)
retriever.get_relevant_documents(query)

In [ ]:
!pip install huggingface-hub



In [ ]:
!pip install transformers

In [ ]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
    repo_id="BioMistral/BioMistral-7B-GGUF",
    filename="ggml-model-Q4_K_M.gguf",
    temperature=0.5,
    max_tokens=512,
    top_p=1
)


In [23]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate
template = """
<|context|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
<|user|>
{query}
</s>
 <|assistant|>
"""
prompt = ChatPromptTemplate.from_template(template)
rag_chain = (
    {'context': retriever, 'query': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
from langchain.prompts import ChatPromptTemplate

# Define the prompt template with clear instructions.
prompt_template = """
Context:
{context}

Question:
{query}

Instructions:
Based on the above context (which may include overlapping or redundant text), please provide a concise and clear answer to the question.
Summarize and deduplicate any repeated information. Focus only on the key details relevant to the question.
Answer:
"""
chat_prompt = ChatPromptTemplate.from_template(prompt_template)

# Define your query
query = "What radiotherapy treatment was recommended for the patient with brain metastases?"

# Retrieve documents from the vector store retriever
retrieved_docs = retriever.get_relevant_documents(query)

# Combine the retrieved text (using a set for deduplication)
context = "\n\n".join({doc.page_content for doc in retrieved_docs})

# Format the prompt
formatted_prompt = chat_prompt.format(context=context, query=query)

# Generate a response using your LLM (e.g., llm, loaded using Llama.from_pretrained)
response = llm(formatted_prompt)
print("Answer:", response)
